In [ ]:
import gradio as gr
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List, Optional
import os
import requests
import json

os.environ["GROQ_API_KEY"] = "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"
os.environ["SERPAPI_API_KEY"] = "bd0d819e093e6c8acf32d6aaab551208bb2f15b199966fa215daa790ba73c33d"

class SearchResult(BaseModel):
    answer: str = Field(..., description="The summarized answer to the user's query")
    sources: List[str] = Field(default_factory=list, description="List of source URLs used for the answer")

llm = ChatGroq(
    api_key=os.environ["GROQ_API_KEY"],
    model_name="llama3-70b-8192",
    temperature=0.7
)

def web_search(query: str) -> str:
    api_key = os.environ["SERPAPI_API_KEY"]
    params = {
        "engine": "google",
        "q": query,
        "api_key": api_key
    }
    try:
        response = requests.get("https://serpapi.com/search", params=params)
        response.raise_for_status()
        data = response.json()
        results = data.get("organic_results", [])
        if not results:
            return json.dumps({"answer": f"No results found for: {query}", "sources": []})
      
        formatted_results = [
            {"title": r.get("title"), "snippet": r.get("snippet"), "link": r.get("link")}
            for r in results[:3]
        ]
        summary_prompt = PromptTemplate(
            input_variables=["results", "query"],
            template="Based on the following search results, provide a concise answer to the query '{query}':\n{results}"
        )
        summary_chain = summary_prompt | llm
        summary = summary_chain.invoke({
            "results": "\n".join([f"- {r['title']}: {r['snippet']} ({r['link']})" for r in formatted_results]),
            "query": query
        }).content
        return json.dumps({
            "answer": summary,
            "sources": [r["link"] for r in formatted_results]
        })
    except Exception as e:
        return json.dumps({"answer": f"Error fetching search results: {str(e)}", "sources": []})

tools = [
    Tool(
        name="Web Search",
        func=web_search,
        description="Performs a web search using SerpAPI to fetch relevant information for a given query."
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

def process_query(user_input):
    try:
        if not user_input.strip():
            return "Please enter a valid query."
        # Run the agent with the user's query
        raw_response = agent.run(
            f"Answer the following query with a concise response and include source URLs if web search is used. Format the output as JSON with 'answer' and 'sources' fields: {user_input}"
        )
        try:
            result = json.loads(raw_response)
            search_result = SearchResult(**result)
           
            output = f"**Answer**: {search_result.answer}\n\n**Sources**:\n" + "\n".join([f"- {url}" for url in search_result.sources])
            return output
        except json.JSONDecodeError:
            return f"Error parsing agent response: {raw_response}"
    except Exception as e:
        return f"Error processing request: {str(e)}"

with gr.Blocks(title="Intelligent Web Search Agent") as demo:
    gr.Markdown("# Intelligent Web Search Agent")
    gr.Markdown("Enter any query, and the agent will search the web to provide a concise answer with source URLs.")
    
    user_input = gr.Textbox(
        label="Enter your query",
        placeholder="e.g., What is the capital of France? or Latest AI news"
    )
    
    submit_button = gr.Button("Submit")
    output = gr.Textbox(label="Agent Response", lines=10)
    
    submit_button.click(
        fn=process_query,
        inputs=user_input,
        outputs=output
    )

if __name__ == "__main__":
    demo.launch()

c:\Users\MITHUN\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\MITHUN\AppData\Local\Temp\ipykernel_28564\3687390117.py:67: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = in

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


C:\Users\MITHUN\AppData\Local\Temp\ipykernel_28564\3687390117.py:79: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  raw_response = agent.run(




> Entering new AgentExecutor chain...
Thought: I need to understand what Langchain is, so I'll perform a web search to get relevant information.

Action: Web Search
Action Input: "what is langchain"

Observation: {"answer": "LangChain is an open-source framework for developing applications powered by large language models (LLMs), simplifying every stage of the LLM application lifecycle.", "sources": ["https://www.langchain.com/", "https://python.langchain.com/docs/introduction/", "https://aws.amazon.com/what-is/langchain/"]}
Thought:Thought: I have the observation from the web search, now I need to format the output as JSON with 'answer' and 'sources' fields.

Final Answer: {"answer": "LangChain is an open-source framework for developing applications powered by large language models (LLMs), simplifying every stage of the LLM application lifecycle.", "sources": ["https://www.langchain.com/", "https://python.langchain.com/docs/introduction/", "https://aws.amazon.com/what-is/langchain/"